In [ ]:
import os
import requests

In [ ]:
from dotenv import load_dotenv 
from IPython.display import Markdown, display 
from bs4 import BeautifulSoup
from openai import OpenAI 

In [ ]:
#-------------------------Method 1 (Useful for quick testing, not recommended for production):---------------------------------------------------

#openai_api_key = "YOUR_OPENAI_API_KEY"

#-------------------------Method 2 - RECOMMENDED (Load API key from .env file, keeps your keys private and safe):--------------------------------
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key (GPT) exists and begin with {openai_api_key[:7]}")
else:
    print("OpenAI API Key not found. Please add your key in the .env file in the root directory.")

In [ ]:
openai = OpenAI()

In [ ]:
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36" }

class Website:

    def __init__(self,url):
        self.url = url;
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img" ,"input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip = True)       

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):  
    user_prompt = f"You are looking at a website titled {website.title}"  
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"  
    user_prompt += website.text  
    return user_prompt 

In [ ]:
def messages_for(website):  
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
def display_summary(url):
    website = Website(url)
    response = openai.chat.completions.create(  
        model="gpt-4o-mini",  
        messages=messages_for(website)  
    )
    summary = response.choices[0].message.content  
    display(Markdown(summary))  

In [ ]:
display_summary("https://github.com/cristi-bogdan")